In [1]:
# import libraries
import requests
import boto3
import sagemaker.amazon.common as smac
import re
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder

In [2]:
# specify bucket name
s3_bucket_name = 'estellaliuml'
bucket_name = 'estellaliuml'
training_folder = r'model/training/'
validation_folder = r'model/validation/'
test_folder = r'model/test/'

s3_model_output_location = r's3://{0}/model/model'.format(bucket_name)
s3_model_training_file_location = r"s3://{0}/{1}".format(bucket_name,training_folder)
s3_validation_file_location = r"s3://{0}/{1}".format(bucket_name,validation_folder)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_folder)

In [3]:
# import the dataset
url = "https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.csv"

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

download_file(url)

'index.csv'

In [3]:
tfr = pd.read_csv('index.csv', skiprows=5)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11,29,30,31,32,33,34,35,37,40,41,42,43,44,45,46,51,54,57,58,82,83,85,86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# trim dataset
temp = tfr.loc[(tfr.Unit=="Hrs")]
newdata = temp[["EffectiveDate","PricePerUnit","Location","Instance Type", "Operating System"]]

# data quality check
newdata.isnull().sum()
newdata2 = newdata.dropna()

In [5]:
# get the unique values
for col in newdata2[["Location","Instance Type","Operating System"]]:
    print(pd.unique(newdata2[col]))

# find the max and min value
print("max value is" + newdata2["EffectiveDate"].max())
print("min value is" + newdata2["EffectiveDate"].min())

# backfill missing observations
L=pd.unique(newdata2["Location"])
I=pd.unique(newdata2["Instance Type"])
O=pd.unique(newdata2["Operating System"])
E=pd.unique(newdata2["EffectiveDate"])
mux = pd.MultiIndex.from_product([L,I,O,E],names=["Location","Instance Type","Operating System","EffectiveDate"])
gdata = newdata.groupby(["Location","Instance Type","Operating System","EffectiveDate"]).mean().reset_index()
gdata = gdata.set_index(["Location","Instance Type","Operating System","EffectiveDate"]).reindex(mux).groupby(level=0).bfill().reset_index()
gdata = gdata.sort_values(by=["Location","Instance Type","Operating System","EffectiveDate"])
gdata["EffectiveDate"]= pd.to_datetime(gdata["EffectiveDate"])

['US East (Ohio)' 'US West (Oregon)' 'AWS GovCloud (US-West)'
 'US East (N. Virginia)' 'Asia Pacific (Tokyo)' 'EU (London)'
 'Middle East (Bahrain)' 'US West (N. California)'
 'AWS GovCloud (US-East)' 'Asia Pacific (Osaka)' 'Asia Pacific (Seoul)'
 'Asia Pacific (Sydney)' 'Canada (Central)' 'EU (Frankfurt)' 'EU (Milan)'
 'Asia Pacific (Singapore)' 'EU (Ireland)' 'Asia Pacific (Hong Kong)'
 'South America (Sao Paulo)' 'EU (Stockholm)' 'US West (Los Angeles)'
 'EU (Paris)' 'Asia Pacific (Mumbai)' 'Africa (Cape Town)'
 'US West (Verizon) - Las Vegas' 'US East (Philadelphia)'
 'US West (Verizon) - Denver' 'US West (Verizon) - Phoenix'
 'US West (Denver)' 'US East (Verizon) - Boston'
 'US West (Verizon) - Seattle' 'US East (Verizon) - Miami'
 'US East (Verizon) - Dallas' 'US East (Houston)'
 'US West (Verizon) - San Francisco Bay Area'
 'US East (Verizon) - Chicago' 'US East (Verizon) - Atlanta'
 'US East (Verizon) - New York' 'US East (Miami)'
 'US East (Verizon) - Houston' 'Asia Pacific (K

In [6]:
# Label Encoder
labelencoder = LabelEncoder()
gdata_cols=["Location","Instance Type","Operating System"]
attach = []
for col in gdata_cols:
    gdata[col] = labelencoder.fit_transform(gdata[col])
    
print(gdata["Location"].max(),gdata["Location"].min(),gdata["Instance Type"].max(),gdata["Operating System"].max())   

46 0 460 4


In [7]:
def add_features(df):
    df['year'] = df["EffectiveDate"].dt.year
    df['month'] = df["EffectiveDate"].dt.month
    df['day'] = df["EffectiveDate"].dt.day
add_features(gdata)

In [8]:
gdata["Location"]=gdata["Location"].astype(np.int16)

In [9]:
gdata["Instance Type"]=gdata["Instance Type"].astype(np.int16)
gdata["Operating System"]=gdata["Operating System"].astype(np.int16)
gdata["PricePerUnit"]=gdata["PricePerUnit"].astype(np.float16)
gdata["year"]=gdata["year"].astype(np.int16)
gdata["month"]=gdata["month"].astype(np.int16)
gdata["day"]=gdata["day"].astype(np.int16)

In [17]:
gdata.to_csv(r'all_data.csv',index=True,index_label='EffecitveDate',columns=["Location","Instance Type","Operating System",
                                                                           "day","month","year","PricePerUnit"])

In [11]:
cutoff_date1 = gdata["EffectiveDate"].max()-timedelta(days=365)
cutoff_date2 = cutoff_date1-timedelta(days=365)
cutoff_date3 = cutoff_date2-timedelta(days=365)

In [14]:
# Spliting training, testing and validation
testing = gdata[(gdata["EffectiveDate"] > cutoff_date1)]
training = gdata[((gdata["EffectiveDate"] <= cutoff_date1)&(gdata["EffectiveDate"] > cutoff_date3))]

In [15]:
np.random.seed(5)
l = list(training.index)
np.random.shuffle(l)
training = training.loc[l]
rows = training.shape[0]
train = int(.7 * rows)
test = rows-train
training.iloc[:train].to_csv('train.csv'
                          ,index=False,header=False
                          ,columns=["Location","Instance Type","Operating System","day","month","year","PricePerUnit"])
training.iloc[train:].to_csv('validation.csv'
                          ,index=False,header=False
                          ,columns=["Location","Instance Type","Operating System","day","month","year","PricePerUnit"])

In [16]:
# write from s3 bucket
def write_to_s3(filename,bucket,key):
    with open(filename,"rb") as f: # Read in binary mode
        return boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(f)

testing.to_csv(r'test.csv')   

write_to_s3("train.csv", s3_bucket_name, training_folder+"train.csv")
write_to_s3("test.csv", s3_bucket_name, test_folder+"test.csv")
write_to_s3("validation.csv", s3_bucket_name, validation_folder+"validation.csv")